# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [ ]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_lumpy,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.0480525168333334 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_004_CA_1_validation,0.0,1.665022,0.1
1,HOUSEHOLD_1_004_CA_1_validation,1.0,1.665022,0.1
2,HOUSEHOLD_1_004_CA_1_validation,1.0,1.521292,0.1
3,HOUSEHOLD_1_004_CA_1_validation,2.0,1.487854,0.1
4,HOUSEHOLD_1_004_CA_1_validation,2.0,1.521927,0.1
...,...,...,...,...
88906,HOUSEHOLD_2_505_CA_4_validation,4.0,0.190597,0.9
88907,HOUSEHOLD_2_505_CA_4_validation,0.0,2.589066,0.9
88908,HOUSEHOLD_2_505_CA_4_validation,0.0,2.589066,0.9
88909,HOUSEHOLD_2_505_CA_4_validation,0.0,2.589066,0.9


In [ ]:
pattern_df_result_all_params.to_csv('Croston_Lumpy_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.09530143064999948 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_004_CA_1_validation,0.1,0.830735,86.8531,94.0137,56.2916,1.361482,1.809133
1,HOUSEHOLD_1_004_CA_1_validation,0.2,0.872165,91.1847,94.9262,59.2403,1.429382,1.889625
2,HOUSEHOLD_1_004_CA_1_validation,0.3,0.917505,95.9250,95.4056,61.1888,1.503690,1.987762
3,HOUSEHOLD_1_004_CA_1_validation,0.4,0.972926,101.7193,95.7977,63.7257,1.594518,2.114792
4,HOUSEHOLD_1_004_CA_1_validation,0.5,1.039973,108.7289,96.6923,67.8495,1.704400,2.275086
...,...,...,...,...,...,...,...,...
2398,HOUSEHOLD_2_512_CA_2_validation,0.5,1.112621,112.0190,114.3969,75.4911,1.483495,1.882310
2399,HOUSEHOLD_2_512_CA_2_validation,0.6,1.147011,115.4814,116.0138,80.9080,1.529348,1.940560
2400,HOUSEHOLD_2_512_CA_2_validation,0.7,1.167465,117.5406,116.1641,84.0099,1.556619,2.001118
2401,HOUSEHOLD_2_512_CA_2_validation,0.8,1.184057,119.2112,115.7178,85.9842,1.578743,2.065099


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
18,HOUSEHOLD_1_012_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
19,HOUSEHOLD_1_012_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
20,HOUSEHOLD_1_012_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
21,HOUSEHOLD_1_012_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
22,HOUSEHOLD_1_012_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2362,HOUSEHOLD_2_499_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2363,HOUSEHOLD_2_499_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2364,HOUSEHOLD_2_499_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2365,HOUSEHOLD_2_499_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
18,HOUSEHOLD_1_012_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
19,HOUSEHOLD_1_012_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
20,HOUSEHOLD_1_012_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
21,HOUSEHOLD_1_012_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
22,HOUSEHOLD_1_012_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2362,HOUSEHOLD_2_499_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2363,HOUSEHOLD_2_499_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2364,HOUSEHOLD_2_499_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2365,HOUSEHOLD_2_499_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
18,HOUSEHOLD_1_012_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
19,HOUSEHOLD_1_012_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
20,HOUSEHOLD_1_012_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
21,HOUSEHOLD_1_012_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
22,HOUSEHOLD_1_012_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2362,HOUSEHOLD_2_499_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2363,HOUSEHOLD_2_499_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2364,HOUSEHOLD_2_499_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2365,HOUSEHOLD_2_499_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
18,HOUSEHOLD_1_012_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
19,HOUSEHOLD_1_012_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
20,HOUSEHOLD_1_012_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
21,HOUSEHOLD_1_012_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
22,HOUSEHOLD_1_012_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2362,HOUSEHOLD_2_499_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2363,HOUSEHOLD_2_499_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2364,HOUSEHOLD_2_499_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2365,HOUSEHOLD_2_499_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 19.850187265917604 %
Percentage of unexpected values of WMAPE is: 19.850187265917604 %
Percentage of unexpected values of SMAPE is: 18.726591760299627 %
Percentage of unexpected values of MAPE is: 19.850187265917604 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,1.199416,156.433840,133.050867,60.705517,1.319405,1.766437
0.2,1.181971,151.456823,133.376261,62.486580,1.344525,1.789157
0.3,1.185664,150.967211,134.033013,64.582593,1.379662,1.827289
0.4,1.198042,152.166390,134.744350,66.691805,1.416786,1.874553
0.5,1.216995,154.509454,135.478357,68.873837,1.457468,1.929281
0.6,1.241699,157.818176,136.253464,71.153108,1.501634,1.991582
0.7,1.271789,162.095270,137.007550,73.515074,1.549587,2.063443
0.8,1.309306,167.668306,137.774231,76.139133,1.603476,2.148556
0.9,1.357822,175.247398,138.528102,79.163168,1.666373,2.253656


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.3
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.874778,107.64250,126.95415,58.64385,1.131158,1.513705
0.2,0.889216,111.46605,127.74115,61.43040,1.189857,1.532206
0.3,0.908781,114.23545,129.39645,63.51365,1.204838,1.561515
0.4,0.926052,119.01530,130.18585,64.33795,1.232953,1.597390
0.5,0.952585,122.74245,131.26420,66.23915,1.281159,1.659770
0.6,0.976439,126.90840,132.20415,66.94225,1.326717,1.718445
0.7,1.003858,130.96380,133.73080,69.01305,1.375460,1.785001
0.8,1.030086,135.48005,135.41905,70.98900,1.400103,1.878121
0.9,1.065456,140.99790,136.85350,74.64060,1.452390,1.973174


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.1
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.1

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_lumpy, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0475362127833345 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_004_CA_1_validation,0.832726,107.6139,113.2837,45.4415,1.153006,1.313869
1,HOUSEHOLD_1_004_CA_2_validation,inf,inf,200.0000,NaN,1.269452,1.269452
2,HOUSEHOLD_1_012_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOUSEHOLD_1_032_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
4,HOUSEHOLD_1_032_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
262,HOUSEHOLD_2_499_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
263,HOUSEHOLD_2_505_CA_4_validation,1.645705,236.3063,172.8571,53.4281,0.506371,0.509679
264,HOUSEHOLD_2_506_CA_2_validation,2.389289,257.3080,187.1317,70.8094,0.367583,0.394282
265,HOUSEHOLD_2_510_CA_2_validation,0.809403,80.4613,100.2041,44.4419,1.494282,2.091346


In [ ]:
df_result_final.isnull().sum()

Product     0
MASE       50
WMAPE      50
SMAPE      53
MAPE       67
MAE         0
RMSE        0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Lumpy_Test_Data.csv')